<a href="https://colab.research.google.com/github/JaroslavHolecek/Teaching/blob/master/JupyterNotebook/SQL/Princip_relacni_databaze_a_pripojeni_zadani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relační databáze — princip a připojení

**Relační databáze** patří mezi nejrozšířenější typ databází. Jejich základem jsou **tabulky**, které jsou vzájemně propojeny předem definovanými **vztahy** (relacemi). Propojení prakticky znamená, že se hodnoty z jedné tabulky odkazují na hodnoty v jiné tabulce.

Tabulky obsahují:
- **sloupce** — nazýváme je **atributy** (určují, jaké vlastnosti si o entitě ukládáme),
- **řádky** — nazýváme je **záznamy** (obsahují konkrétní data).

---

## Entita

V reálném světě máme objekty (např. auto, dům, člověk …) i abstraktní pojmy (např. řidičská kvalifikace, barva, objednávka …). Souhrnně tyto „věci" nazýváme **entita**.

Každá entita má své **vlastnosti** — auto má značku, SPZ, počet kol, počet dveří atd.

Atributy tabulky určují, které vlastnosti entity si budeme ukládat. Konkrétní data — např. `"BMW", "2A5 0101", 4, 5` — jsou potom záznamy (řádky) v tabulce.

---

## Příklady relačních databázových systémů (RDBMS)

| Systém | Licence | Poznámka |
|---|---|---|
| **MySQL** | Open-source / komerční | Nejrozšířenější open-source RDBMS |
| **PostgreSQL** | Open-source | Pokročilé funkce, dodržuje SQL standard |
| **MariaDB** | Open-source | Fork MySQL, plně kompatibilní |
| **SQLite** | Public domain | Souborová databáze, nevyžaduje server |
| **MS SQL Server** | Komerční | Využívaný v enterprise prostředí |
| **Oracle DB** | Komerční | Robustní, enterprise řešení |

---

## SŘBD — Systém řízení báze dat

SŘBD (anglicky **DBMS** — Database Management System) je program/aplikace, který se stará o správu dat. Jeho úkolem je zajistit, aby data v databázi byla konzistentní a správná.

SŘBD například:
- hlídá **datové typy** (do sloupce s číslem nelze zapsat text),
- **indexuje** záznamy pro rychlejší vyhledávání,
- hlídá **unikátní hodnoty** ve sloupcích (pokud tak tabulku nastavíme — `UNIQUE`),
- hlídá **referenční integritu** napříč tabulkami pomocí vlastních a cizích klíčů (`PRIMARY KEY`, `FOREIGN KEY`),
- řídí **transakce** (zajišťuje, že se buď provedou všechny operace, nebo žádná),
- spravuje **přístupová práva** uživatelů,
- a mnoho dalšího.

---

## Atribut (sloupec)

Sloupec v tabulce určuje, jakou informaci si budeme o entitě v databázi pamatovat. Abychom správně nastavili atribut, musíme mu přiřadit:

1. **Název** — měl by být výstižný, psaný bez diakritiky a mezer (používáme `snake_case`), např. `datum_narozeni`, `pocet_dveri`.
2. **Datový typ** — např. `INT` (celé číslo), `VARCHAR(n)` (text do _n_ znaků), `DATE` (datum), `FLOAT` / `DECIMAL` (desetinné číslo), `BOOLEAN` (pravda/nepravda).
3. **Omezení (constraints)** — volitelné vlastnosti: `NOT NULL`, `UNIQUE`, `DEFAULT`, `CHECK` apod.

> **Pravidlo:** Do každého atributu lze zapsat pouze **jednu hodnotu** na každém řádku. Pokud potřebujeme uložit např. 2 telefonní čísla k jednomu člověku, správné řešení je vytvořit **novou tabulku** (ne druhý sloupec — to porušuje normalizaci).

---

## Atomicita atributů

Atribut by měl být **atomický** — to znamená, že hodnota v něm uložená už nejde smysluplně rozložit na menší části.

**Příklad:**

❌ Neatomický atribut: `jmeno_a_prijmeni` → `"Pavel Novotný"` (lze rozdělit)

✅ Atomické atributy: `jmeno` → `"Pavel"`, `prijmeni` → `"Novotný"` (nelze dále rozdělit)

Správná volba atomických atributů nám zajistí hladký chod databáze, jednoduché používání a efektivní dotazování.

---

## Primární klíč (PRIMARY KEY)

Každá tabulka by měla mít **primární klíč** — atribut (nebo kombinaci atributů), který **jednoznačně identifikuje** každý záznam v tabulce. Nejčastěji se používá sloupec `id` s automatickým číslováním (`AUTO_INCREMENT`).

Primární klíč:
- musí být **unikátní** (žádné dva řádky nemají stejnou hodnotu),
- nesmí být **NULL** (musí mít vždy hodnotu),
- v tabulce může být **pouze jeden**.

---

## Ukázka tabulky

| | `id` | `jmeno` | `prijmeni` | `datum_narozeni` |
|---|---|---|---|---|
| **Záznam 1** | 1 | Pavel | Novotný | 1990-01-01 |
| **Záznam 2** | 2 | Jakub | Novák | 2002-07-10 |

> **Konvence:** Názvy tabulek a atributů píšeme **malými písmeny**, bez diakritiky, s podtržítky místo mezer (`snake_case`). Datum ukládáme ve formátu `YYYY-MM-DD` (ISO 8601).

---

**Pozn.:** Atributy, relace a pokročilejší témata (normální formy, JOIN, …) se budou podrobněji probírat v dalších podkladech.

# Připojení do školní databáze pomocí Pythonu

K práci s MySQL databází z Pythonu použijeme knihovnu **`mysql-connector-python`** (oficiální connector od Oracle). Nejdříve ji musíme nainstalovat:

> **Pozn.:** Balíček `mysql-connector` (bez `-python`) je zastaralý a může způsobovat problémy. Vždy instalujte `mysql-connector-python`.

In [ ]:
# Instalace knihovny (stačí spustit jednou)
! pip install mysql-connector-python


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Připojení a odpojení

Nyní k samotnému připojení. Postup má vždy 3 kroky:

1. **Připojení** k databázi pomocí `mysql.connector.connect()`
2. **Vytvoření kurzoru** — objekt, přes který posíláme SQL příkazy a čteme výsledky
3. **Odpojení** — vždy na konci uzavřeme kurzor i spojení

> **Důležité:** Přihlašovací údaje (heslo, uživatel) byste **nikdy neměli psát přímo do kódu**. V praxi se ukládají do konfiguračních souborů nebo proměnných prostředí. My je máme v souboru `pripojeni.py`.

In [ ]:
import mysql.connector
from pripojeni import *  # importuje konstanty HOST, USER, PASSWORD, DATABASE

# 1) Připojení k databázi
mydb = mysql.connector.connect(
    host=HOST,          # URL adresa databázového serveru
    user=USER,          # uživatelské jméno
    password=PASSWORD,  # heslo
    database=DATABASE   # název databáze
)

# 2) Vytvoření kurzoru — přes něj posíláme SQL příkazy a čteme výsledky
mycursor = mydb.cursor()

# ... zde budeme později psát SQL dotazy ...

# 3) Odpojení — vždy na konci uzavřeme kurzor i spojení
mycursor.close()
mydb.close()

print("✅ Připojení a odpojení proběhlo úspěšně.")

## Proč je důležité se odpojovat?

Každé připojení k databázi spotřebovává **prostředky na serveru** (paměť, síťový socket, vlákno). Pokud se neodpojíte:

1. **Vyčerpání limitu spojení** — MySQL má nastavený maximální počet současných připojení (např. 150). Neuzavřená spojení zůstávají otevřená a server časem odmítne nová.
2. **Únik paměti** — každé spojení zabírá RAM na serveru i v Pythonu. Při opakovaném spouštění buněk v notebooku se spojení hromadí.
3. **Zamčené záznamy** — nedokončená transakce může držet zámek na řádcích/tabulce a blokovat ostatní uživatele.
4. **Ztráta dat** — nepotvrzené změny (bez `commit()`) se při násilném odpojení zahodí.

> **⚠️** Ve školním prostředí je to obzvlášť důležité — na jednom serveru pracuje celá třída najednou.

**Best practice:** V produkčním kódu se používá `try/finally`, aby se spojení zavřelo i při chybě:

```python
try:
    mydb = mysql.connector.connect(...)
    mycursor = mydb.cursor()
    # ... práce s DB ...
    mydb.commit()
finally:
    mycursor.close()
    mydb.close()
```

# Cvičení

## Cvičení 1 — Připojení k databázi
Napište kód pro připojení a následné odpojení od školní databáze. Použijte konstanty z modulu `pripojeni`.

<details>
<summary>🔎 Očekávaný výstup (klikni pro zobrazení)</summary>

```
✅ Připojení a odpojení proběhlo úspěšně.
```

</details>


In [ ]:
# TODO — Cvičení 1: Napište kód pro připojení a odpojení od školní databáze
